### Import libraries

In [ ]:
import glob
import numpy as np
import pandas as pd
import sklearn 
import seaborn as sns
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
from skimage.io import imread
from copy import deepcopy 

import tensorflow.compat.v2 as tf 
import tensorflow.keras as keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, UpSampling2D
from tensorflow.keras.optimizers import Adam

Set path to images

In [ ]:
PATH_TO_IMG = "../input/celeba-dataset/img_align_celeba/img_align_celeba/"

path_to_img_data = glob.glob(PATH_TO_IMG + "*jpg")
len(path_to_img_data)

Set size of dataset, batch size and shape of image

In [ ]:
TRAIN_SIZE = 25600
TEST_SIZE = 640

BATCH_SIZE = 128

IMG_HEIGHT = 224
IMG_WIDTH = 184

Plot original image

In [ ]:
plt.figure(figsize=(10,5))
for i,img_path in enumerate(path_to_img_data[:4]):
    plt.subplot(1,4,i+1)
    plt.axis('off')
    img = plt.imread(img_path)
    plt.imshow(img)   

My generator, input image in grayscale, output colorated images.

In [ ]:
def get_input(image_path):
    img = imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)      
    img = img/255
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))

    return img 


def get_output(image_path):
    img = imread(image_path)
    img = img/255.0
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    return img
    
    
def image_generator(files, batch_size = 64): 
    
    tmp_x = deepcopy(files)
    
    while True:
        len_array = len(tmp_x)
        if len_array < batch_size:
          tmp_x = deepcopy(files)
          len_array = len(tmp_x)

        batch_index = np.random.choice(len_array, size=batch_size, replace=False)
        batch_paths = [tmp_x[index] for index in batch_index]
        tmp_x = np.delete(tmp_x, batch_index)
        batch_input  = []
        batch_output = [] 
        for input_path in batch_paths:
            input_img = get_input(input_path)
            output = get_output(input_path)
            batch_input.append(input_img)
            batch_output.append(output)
        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )

        yield( batch_x, batch_y )

Train and test generator with TRAIN/TEST size + BATCH size 

In [ ]:
train_gen = image_generator(path_to_img_data[:TRAIN_SIZE],BATCH_SIZE)
test_gen = image_generator(path_to_img_data[TRAIN_SIZE:TRAIN_SIZE+TEST_SIZE],BATCH_SIZE)

First test generation of data

In [ ]:
plot_example_data = next(test_gen)

Plotting input and output data

In [ ]:
plt.imshow(plot_example_data[0][1].reshape(IMG_HEIGHT,IMG_WIDTH),cmap='gray')
plt.show()

In [ ]:
plt.imshow(plot_example_data[1][1])
plt.show()

Model of convolutional autoencoder

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()

Train model

In [ ]:
STEP = TRAIN_SIZE//BATCH_SIZE
EPOCHS = 25
model.fit_generator(generator=train_gen,steps_per_epoch=STEP,epochs=EPOCHS)

In [ ]:
test_img = next(test_gen)

Plot - original data, input data, output data

In [ ]:
predicted = model.predict(test_img[0])

n = 5
plt.figure(figsize=(15, 10))
for i in range(n):
    # display original
    plt.subplot(3, n, i + 1 )
    plt.imshow(test_img[1][i].reshape(IMG_HEIGHT, IMG_WIDTH, 3))
    plt.axis('off')
    
    
    # display grayscale
    plt.subplot(3, n, i + 1 + n )
    plt.imshow(test_img[0][i].reshape(IMG_HEIGHT, IMG_WIDTH), cmap='gray')
    plt.axis('off')
    
    # display predict
    plt.subplot(3, n, i + 1 + n * 2 )
    plt.imshow(predicted[i].reshape(IMG_HEIGHT, IMG_WIDTH, 3))
    plt.axis('off')
        
plt.show()